In [8]:
import tensorflow as tf
import os

In [9]:
gpus = tf.config.experimental.list_physical_devices("GPUS")
len(gpus)

0

In [11]:
import cv2
import imghdr

C:\Users\User\AppData\Local\Temp\ipykernel_23816\4232469594.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr
